In [48]:
from google.cloud import bigquery
import pandas as pd
import json

from datetime import date,datetime,timedelta

In [49]:
table_id = "pongthorn.SMartML.new_incident"
mapping_file="incident_sevirity_to_class.json"
PATH_FOLDER_ARTIFACTS="model"
unUsedColtoPredict=['severity','id','severity_id','severity_name','imported_at']

In [50]:

with open(mapping_file, 'r') as json_file:
     map_sevirity_to_class= json.load(json_file)
print(map_sevirity_to_class)

{'Cosmatic': 0, 'Minor': 1, 'Major': 2, 'Critical': 3}


In [51]:
# Get today's date
today = date.today()
# Yesterday date
yesterday = today - timedelta(days = 1)
str_today=today.strftime('%Y-%m-%d')
str_yesterday=yesterday.strftime('%Y-%m-%d')
print(f"Get data between {str_yesterday} to {str_today} to predict sevirity level")

Get data between 2023-03-27 to 2023-03-28 to predict sevirity level


In [52]:
def load_data_bq(sql:str):
 client_bq = bigquery.Client()
 query_result=client_bq.query(sql)
 df=query_result.to_dataframe()
 return df

In [53]:
sql=f"""
SELECT *  FROM `{table_id}` 
WHERE DATE(imported_at) >= '{str_yesterday}' and DATE(imported_at) < '{str_today}'  
LIMIT 5
"""

dfNewData=load_data_bq(sql)
dfNewData=dfNewData.drop_duplicates(subset=['id'],keep='first')

dfNewData.insert(2, 'severity', dfNewData['severity_name'].map(map_sevirity_to_class),True)


print(dfNewData.info())
dfNewData

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         5 non-null      Int64         
 1   severity_id                5 non-null      Int64         
 2   severity                   5 non-null      int64         
 3   severity_name              5 non-null      object        
 4   sla                        5 non-null      object        
 5   product_type               5 non-null      object        
 6   brand                      5 non-null      object        
 7   service_type               5 non-null      object        
 8   incident_type              5 non-null      object        
 9   open_to_close_hour         5 non-null      float64       
 10  response_to_resolved_hour  5 non-null      float64       
 11  imported_at                5 non-null      datetime64[ns]
dtypes: Int64(2),

,id,severity_id,severity,severity_name,sla,product_type,brand,service_type,incident_type,open_to_close_hour,response_to_resolved_hour,imported_at
0,2452,2,2,Major,24x7 4Hrs Response Time,Server,HPE,Incident,Software,4.500000,4.0,2023-03-27 05:27:08.968019
1,2517,2,2,Major,24x7 4Hrs Resolution Time,Server,HPE,Incident,Memory Failure,6.166667,6.0,2023-03-27 05:27:08.968019
2,2519,2,2,Major,24x7 4Hrs Resolution Time,Server,HPE,Incident,Hard Disk Drive Failure,7.166667,7.0,2023-03-27 05:27:08.968019
3,2518,2,2,Major,24x7 4Hrs Resolution Time,Server,HPE,Incident,Hard Disk Drive Failure,7.166667,7.0,2023-03-27 05:27:08.968019
4,2520,2,2,Major,24x7 4Hrs Resolution Time,Server,HPE,Incident,Hard Disk Drive Failure,7.166667,7.0,2023-03-27 05:27:08.968019


In [54]:
model = tf.keras.models.load_model(PATH_FOLDER_ARTIFACTS)    
print(f"Load from {PATH_FOLDER_ARTIFACTS}")
print(model.summary())

Load from model
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sla (InputLayer)               [(None, 1)]          0           []                               
                                                                                                  
 product_type (InputLayer)      [(None, 1)]          0           []                               
                                                                                                  
 brand (InputLayer)             [(None, 1)]          0           []                               
                                                                                                  
 service_type (InputLayer)      [(None, 1)]          0           []                               
                                                                            